In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from sklearn.linear_model import LogisticRegression

from collections import Counter
import joblib


import data_splitter as ds
import data_loader as dl
import run_model as rm

2023-04-11 23:27:50.335254: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-11 23:27:54.872568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 23:28:07.766065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV

In [2]:
KERASTUNER_TUNER_ID="chief"
KERASTUNER_ORACLE_IP="127.0.0.1"
KERASTUNER_ORACLE_PORT="8000"

In [3]:
%load_ext autoreload
%autoreload 2

# Hyperparameter search
Using 25 % of the training set and the PCA model trained on fold 0 as feature extractor

## Image resolution 256 x 256

In [ ]:
BASEPATH = '' # directory of images preprocessed as numpy arra

# scitkit-learn parameter search does not support batch learning
# so we'll have to combine data in 1 batch
BATCH_SIZE = 256 

# from PCA, image resolution does not seem to have any effect on no. of component
# so we'll use the lowest resolution to minimize training cost
IMG_SIZE = (256, 256, 1)

RANDOM_STATE = 42

# define DataGenerators for training split
train_gen = dl.DataGenerator(
    list_IDs = sample_img_ids,
    labels = splitter.labels,
    patient_img_dict = splitter.trainset,
    basepath = BASEPATH,
    batch_size = BATCH_SIZE,
    img_size = IMG_SIZE,
    from_numpy= True,
    
    shuffle = True,
    normalize = (0, 1),
    
    feature_extractor = 'CNN',
    CNN_preprocess=tf.keras.applications.inception_v3.preprocess_input,
    verbose = True
    )

# check no. of batch
print(len(train_gen))

In [ ]:
# pre-trained models for feature extraction
IMG_SIZE = (256, 256, 1)

inception = keras.applications.InceptionV3(
    weights='imagenet',  # load weights pre-trained on ImageNet.
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),  # should have exactly 3 inputs channels because it is pre-trained on RBG images
    include_top=False
    )

inception.trainable = False
    
inception.summary()

In [ ]:
# get X and y
for i, (X_batch, y_batch) in enumerate(train_gen):
    
    X_batch = inception(X_batch, training=False)
    print(X_batch.shape)
    
    if i==0:
        X = X_batch
        y = y_batch
    else:
        X = np.concatenate((X, X_batch), axis=0)
        y = np.concatenate((y, y_batch), axis=0)
    
    print(X.shape, y.shape)

In [ ]:
# with open('sample_inception_imagenet_X.npy','wb') as f:
#     np.save(f, X)
    
# with open('sample_inception_imagenet_y.npy','wb') as f:
#     np.save(f, y)

In [5]:
X = np.load('sample_inception_imagenet_X.npy')
y = np.load('sample_inception_imagenet_y.npy')

In [6]:
X = X.reshape(X.shape[0], -1)
print(X.shape)

(8704, 73728)


In [7]:
logit = LogisticRegression(
    warm_start = True,
    solver = 'saga',
    max_iter = 10000,
    l1_ratio = 0.5,
    random_state = 42
    )

param_grid = {
    'C': np.logspace(0, -4, num=5, base=10, endpoint=True),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'tol': np.logspace(0, -4, num=5, base=10, endpoint=True)
}

In [11]:
grid = rm.hp_search(X, y, logit, param_grid, random_state=42, min_resources=800, verbose=1)

#save your model or results
joblib.dump(grid, '../hyperparameter_search/cnn_imagenet_logit_hp_search_256_randomstate42_minres800.pkl')

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 800
max_resources_: 8704
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 10
n_resources: 800
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/l

----------
iter: 1
n_candidates: 2
n_resources: 4000
Fitting 5 folds for each of 2 candidates, totalling 10 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/l

Best hyperparameters:  {'tol': 1.0, 'penalty': 'l2', 'C': 0.01}
Best average precision score:  0.02409164343668614


['models/cnn_imagenet_logit_hp_search_256_randomstate42_minres800.pkl']

In [ ]:
grid = rm.hp_search(X, y, logit, param_grid, random_state=42, min_resources=1600, verbose=1)

#save your model or results
joblib.dump(grid, '../hyperparameter_search/cnn_imagenet_logit_hp_search_256_randomstate42_minres1600.pkl')

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 1600
max_resources_: 8704
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 5
n_resources: 1600
Fitting 5 folds for each of 5 candidates, totalling 25 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/l

In [ ]:
grid = rm.hp_search(X, y, logit, param_grid, random_state=0, min_resources=800, verbose=1)

#save your model or results
joblib.dump(grid, '../hyperparameter_search/cnn_imagenet_logit_hp_search_256_randomstate0_minres800.pkl')

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 800
max_resources_: 8704
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 10
n_resources: 800
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/l

----------
iter: 1
n_candidates: 2
n_resources: 4000
Fitting 5 folds for each of 2 candidates, totalling 10 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


# Training Logistic regression
Using 5-fold cross validation

Evaluation metrics:
- **Average precision** the main metric (summarizes a precision-recall curve (PR AUC) as the weighted mean of precisions achieved at each threshold, with the increase in recall from the previous threshold used as the weight)
- F1-score
- Confustion matrix (true positives/false positives/true negatives/false negatives)
- ROC AUC
- Accuracy
- Precision
- Recall
- Brier score loss


In [4]:
metadata = pd.read_csv('train.csv')
len(metadata.image_id.unique())

54706

In [5]:
splitter = ds.DataSplitter(verbose=True)

Total patient_id in training set:  9530
Total patient_id in test set:  2383
Total image_id in training set:  43767
Total image_id in test set:  10939
Total patient_id in training set:  7624
Total patient_id in calibration set:  1906
Total image_id in training set:  35003
Total image_id in calibration set:  8764


In [6]:
# image id of all sample training set after split to calib and train set

train_img_ids = [id for k, v in splitter.trainset.items() for id in v]
print(len(train_img_ids))

35003


In [7]:
# splitter.trainset is the training set in format of {patient_id: [image_id, ...]}
train_ids = splitter.trainset.keys()
label_by_img = splitter.labels

# getting cross-validation folds from training set
RANDOM_STATE = 42
cv_patient, cv_img = splitter.get_cv(n_splits=5, random_state=RANDOM_STATE)

Splitting training set into 5 stratified k-folds...
--Fold:  0
----Total patient_id in training set:  6099
----Total image_id in training set:  28031
----Total patient_id in validation set:  1525
----Total image_id in validation set:  6972
--Fold:  1
----Total patient_id in training set:  6099
----Total image_id in training set:  28016
----Total patient_id in validation set:  1525
----Total image_id in validation set:  6987
--Fold:  2
----Total patient_id in training set:  6099
----Total image_id in training set:  27937
----Total patient_id in validation set:  1525
----Total image_id in validation set:  7066
--Fold:  3
----Total patient_id in training set:  6099
----Total image_id in training set:  28025
----Total patient_id in validation set:  1525
----Total image_id in validation set:  6978
--Fold:  4
----Total patient_id in training set:  6100
----Total image_id in training set:  28003
----Total patient_id in validation set:  1524
----Total image_id in validation set:  7000


## Image resolution 256 x 256

**Best hyperparameters**: 
- C = 0.01
- penalty = l2
- tol = 1 

**Constant hyperparameters**:
- warm_start = True,
- solver = 'saga',
- max_iter = 10000,
- l1_ratio = 0.5,
- random_state = 42

In [17]:
BATCH_SIZE = 488 # losing as less samples as possible
IMG_SIZE = (256, 256, 1)

BASEPATH = '' # directory of images preprocessed as numpy arrays
EXTRACTOR_PATH = '../feature_extractors' # directory to save trained models
SAVEPATH = '../predictive_models'

RANDOM_STATE = 42
HP = {'C': 0.01, 'penalty': 'l2', 'tol': 1, 'class_weight': None}


imagenet_logit_256 = rm.run_cv_training(
    
        model_name = 'logit',
        model_params = HP,
    
        cv_img = cv_img,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
        
        from_numpy = True,
        img_basepath = BASEPATH,
        batch_size = BATCH_SIZE, 
        img_size = IMG_SIZE,
        shuffle = True,
        normalize = (0, 1),

        feature_extractor_name = 'CNN_imagenet',
        CNN_preprocess = tf.keras.applications.inception_v3.preprocess_input,
        n_components = 'output',
        extractor_path = EXTRACTOR_PATH,
        savepath = SAVEPATH,
        random_state = RANDOM_STATE,
 
        verbose = 1,
        return_model = False
        )

Fold:  0
---Feature extractor: CNN with ImageNet weights
---Creating training and validation data generators...
---Initializing logistic regression model...
---Start training model
------Training batch  0
>>> Transforming with CNN...
(488, 6, 6, 2048)


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


------Training batch  1
>>> Transforming with CNN...
(488, 6, 6, 2048)


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


------Training batch  2
>>> Transforming with CNN...
(488, 6, 6, 2048)


...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  12
------Calculating scores of batch  12


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  13
------Calculating scores of batch  13
>>> Finish training model!
>>> Mean scores over batches: 
{'pr_auc': {'mean': 0.03592358842025297, 'std': 0.016390939317207358}, 'brier_loss': {'mean': 0.021462148142557366, 'std': 0.00769172533776197}, 'roc_auc': {'mean': 0.5607020005902918, 'std': 0.11424276344644974}, 'f1-score': {'mean': 0.0, 'std': 0.0}, 'recall': {'mean': 0.0, 'std': 0.0}, 'precision': {'mean': 0.0, 'std': 0.0}, 'confusion_matrix': {'mean': 122.0, 'std': 205.22974930550396}, 'accuracy': {'mean': 0.9781908665105385, 'std': 0.007921580285559027}}
---Saving model...


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Use class weight

In [10]:
pos = metadata[metadata.cancer==1].image_id.nunique()
neg = metadata[metadata.cancer==0].image_id.nunique()
total = metadata[metadata.patient_id.isin(splitter.trainset.keys())].image_id.nunique()

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.33
Weight for class 1: 15.11


In [ ]:
BATCH_SIZE = 931 # to get n_estimators = 30 (1 tree per batch)
IMG_SIZE = (256, 256, 1)

BASEPATH = '' # directory of images preprocessed as numpy arrays
EXTRACTOR_PATH = '../feature_extractors' # directory to load pre-trained feature extractors
SAVEPATH = '../predictive_models/use_class_weight' # directory to save trained models

RANDOM_STATE = 42
HP = {'C': 0.01, 'penalty': 'l2', 'tol': 1, 'class_weight': class_weight}


imagenet_rfc_256 = rm.run_cv_training(
    
        model_name = 'logit',
        model_params = HP,
    
        cv_img = cv_img,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
        
        from_numpy = True,
        img_basepath = BASEPATH,
        batch_size = BATCH_SIZE, 
        img_size = IMG_SIZE,
        shuffle = True,
        normalize = (0, 1),

        feature_extractor_name = 'CNN_imagenet',
        CNN_preprocess = tf.keras.applications.inception_v3.preprocess_input,
        n_components = 'output',
        extractor_path = EXTRACTOR_PATH,
        random_state = RANDOM_STATE,
 
        verbose = 1,
        return_model = False,
        savepath = SAVEPATH
        )

## Image Resolution 512 x 512

In [ ]:
BATCH_SIZE = 488 # losing as less samples as possible
IMG_SIZE = (512, 512, 1)

BASEPATH = '' # directory of images preprocessed as numpy arrays
EXTRACTOR_PATH = '../feature_extractors' # directory containing the feature extractor
SAVEPATH = '../predictive_models' # directory to save traiend models



RANDOM_STATE = 42
HP = {'C': 0.01, 'penalty': 'l2', 'tol': 1, 'class_weight': None}


imagenet_logit_512 = rm.run_cv_training(
    
        model_name = 'logit',
        model_params = HP,
    
        cv_img = cv_img,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
        
        from_numpy = True,
        img_basepath = BASEPATH,
        batch_size = BATCH_SIZE, 
        img_size = IMG_SIZE,
        shuffle = True,
        normalize = (0, 1),

        feature_extractor_name = 'CNN_imagenet',
        CNN_preprocess = tf.keras.applications.inception_v3.preprocess_input,
        n_components = 'output',
        extractor_path = EXTRACTOR_PATH,
        savepath = SAVEPATH,
        random_state = RANDOM_STATE,
 
        verbose = 1,
        return_model = False
        )

Fold:  0
---Feature extractor: CNN with ImageNet weights
---Creating training and validation data generators...
---Initializing logistic regression model...
---Start training model
------Training batch  0
--->>> Tranforming X with CNN...
------Training batch  1
--->>> Tranforming X with CNN...
------Training batch  2
--->>> Tranforming X with CNN...
------Training batch  3
--->>> Tranforming X with CNN...
------Training batch  4
--->>> Tranforming X with CNN...
------Training batch  5
--->>> Tranforming X with CNN...
------Training batch  6
--->>> Tranforming X with CNN...
------Training batch  7
--->>> Tranforming X with CNN...
------Training batch  8
--->>> Tranforming X with CNN...
------Training batch  9
--->>> Tranforming X with CNN...
------Training batch  10
--->>> Tranforming X with CNN...
------Training batch  11
--->>> Tranforming X with CNN...
------Training batch  12
--->>> Tranforming X with CNN...
------Training batch  13
--->>> Tranforming X with CNN...
------Training ba

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  1
------Calculating scores of batch  1


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  2
------Calculating scores of batch  2


...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  1
------Calculating scores of batch  1


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  2
------Calculating scores of batch  2


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
BATCH_SIZE = 488 # losing as less samples as possible
IMG_SIZE = (512, 512, 1)

BASEPATH = '' # directory of images preprocessed as numpy arrays
EXTRACTOR_PATH = '../feature_extractors' # directory containing the feature extractor
SAVEPATH = '../predictive_models' # directory to save traiend models

RANDOM_STATE = 42
HP = {'C': 0.01, 'penalty': 'l2', 'tol': 1, 'class_weight': None}


imagenet_logit_512 = rm.run_cv_training(
    
        model_name = 'logit',
        model_params = HP,
    
        cv_img = cv_img,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
        
        from_numpy = True,
        img_basepath = BASEPATH,
        batch_size = BATCH_SIZE, 
        img_size = IMG_SIZE,
        shuffle = True,
        normalize = (0, 1),

        feature_extractor_name = 'CNN_imagenet',
        CNN_preprocess = tf.keras.applications.inception_v3.preprocess_input,
        n_components = 'output',
        extractor_path = EXTRACTOR_PATH,
        savepath = SAVEPATH,
        random_state = RANDOM_STATE,
 
        verbose = 1,
        return_model = False
        )

Fold:  0
logit model trained on this cross-validation fold already exist
--> /home/irinb/models/CNN_imagenet_output_logit_42_512_0.pkl
Fold:  1
logit model trained on this cross-validation fold already exist
--> /home/irinb/models/CNN_imagenet_output_logit_42_512_1.pkl
Fold:  2
---Feature extractor: CNN with ImageNet weights
---Creating training and validation data generators...
---Initializing logistic regression model...
---Start training model
------Training batch  0
--->>> Tranforming X with CNN...
------Training batch  1
--->>> Tranforming X with CNN...
------Training batch  2
--->>> Tranforming X with CNN...
------Training batch  3
--->>> Tranforming X with CNN...
------Training batch  4
--->>> Tranforming X with CNN...
------Training batch  5
--->>> Tranforming X with CNN...
------Training batch  6
--->>> Tranforming X with CNN...
------Training batch  7
--->>> Tranforming X with CNN...
------Training batch  8
--->>> Tranforming X with CNN...
------Training batch  9
--->>> Tranf

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  1
------Calculating scores of batch  1


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  2
------Calculating scores of batch  2


...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  12
------Calculating scores of batch  12


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  13
------Calculating scores of batch  13
>>> Finish training model!
>>> Mean scores over batches: 
{'pr_auc': {'mean': 0.03512123226307241, 'std': 0.02953620835583863}, 'brier_loss': {'mean': 0.02199114094120422, 'std': 0.005835646246450747}, 'roc_auc': {'mean': 0.4890287140069614, 'std': 0.08398670256913682}, 'f1-score': {'mean': 0.0, 'std': 0.0}, 'recall': {'mean': 0.0, 'std': 0.0}, 'precision': {'mean': 0.0, 'std': 0.0}, 'confusion_matrix': {'mean': 122.0, 'std': 205.34500585251988}, 'accuracy': {'mean': 0.9786299765807963, 'std': 0.006326165920659926}}
---Saving model...


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Image Resolution 1024 x 1024

In [9]:
BATCH_SIZE = 488 # losing as less samples as possible
IMG_SIZE = (1024, 1024, 1)

BASEPATH = '' # directory of images preprocessed as numpy arrays
EXTRACTOR_PATH = '../feature_extractors' # directory containing the feature extractor
SAVEPATH = '../predictive_models' # directory to save traiend models

RANDOM_STATE = 42
HP = {'C': 0.01, 'penalty': 'l2', 'tol': 1, 'class_weight': None}


imagenet_logit_512 = rm.run_cv_training(
    
        model_name = 'logit',
        model_params = HP,
    
        cv_img = cv_img,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
        
        from_numpy = True,
        img_basepath = BASEPATH,
        batch_size = BATCH_SIZE, 
        img_size = IMG_SIZE,
        shuffle = True,
        normalize = (0, 1),

        feature_extractor_name = 'CNN_imagenet',
        CNN_preprocess = tf.keras.applications.inception_v3.preprocess_input,
        n_components = 'output',
        extractor_path = EXTRACTOR_PATH,
        savepath = SAVEPATH,
        random_state = RANDOM_STATE,
 
        verbose = 1,
        return_model = False
        )

Fold:  0
---Feature extractor: CNN with ImageNet weights
---Creating training and validation data generators...
---Initializing logistic regression model...
---Start training model
------Training batch  0
--->>> Tranforming X with CNN...
------Training batch  1
--->>> Tranforming X with CNN...
------Training batch  2
--->>> Tranforming X with CNN...
------Training batch  3
--->>> Tranforming X with CNN...
------Training batch  4
--->>> Tranforming X with CNN...
------Training batch  5
--->>> Tranforming X with CNN...
------Training batch  6
--->>> Tranforming X with CNN...
------Training batch  7
--->>> Tranforming X with CNN...
------Training batch  8
--->>> Tranforming X with CNN...
------Training batch  9
--->>> Tranforming X with CNN...
------Training batch  10
--->>> Tranforming X with CNN...
------Training batch  11
--->>> Tranforming X with CNN...
------Training batch  12
--->>> Tranforming X with CNN...
------Training batch  13
--->>> Tranforming X with CNN...
------Training ba

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  1
------Calculating scores of batch  1


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  2
------Calculating scores of batch  2


...

------Getting predictions for batch  12
------Calculating scores of batch  12


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Getting predictions for batch  13
------Calculating scores of batch  13


/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>>> Finish training model!
>>> Mean scores over batches: 
{'pr_auc': {'mean': 0.05018633274702274, 'std': 0.0351724570914713}, 'brier_loss': {'mean': 0.021169537026260937, 'std': 0.006540482958262632}, 'roc_auc': {'mean': 0.5593393347996582, 'std': 0.1003170865138339}, 'f1-score': {'mean': 0.0, 'std': 0.0}, 'recall': {'mean': 0.0, 'std': 0.0}, 'precision': {'mean': 0.0, 'std': 0.0}, 'confusion_matrix': {'mean': 122.0, 'std': 205.3057823971704}, 'accuracy': {'mean': 0.9784836065573772, 'std': 0.006696328591158007}}
---Saving model...
